In [1]:
"""
    Transfer-Learning is composed 
    VGG, Inception, ResNET, CIFAR-10, ImageNet.
"""

# Image Classification by using CNN-Algorithm.
# Now we are going to use Fashion-MNIST while we train and test this model.
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models

In [2]:
# Create DataSet by applying ImageFolder-Function.
train_imgs = ImageFolder(
    "../04/taco_and_burrito/train/",
    transform=transforms.Compose([
        transforms.RandomCrop(224),
        transforms.ToTensor()
    ])
)

In [3]:
test_imgs = ImageFolder(
    "../04/taco_and_burrito/test/",
    transform=transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
)

In [4]:
# Create DataLoader
train_loader = DataLoader(
    train_imgs, batch_size=32, shuffle=True
)

test_loader = DataLoader(
    test_imgs, batch_size=32, shuffle=False
)

# Increate Layer such as shape of (N, C, H, W) Tensor to the 
# (N, C*H*W).
# It needs for print Convolution-Product.
# class FlatternLayer(nn.Module):
#     def forward(self, x):
#         sizes = x.size()
#         return x.view(sizes[0], -1)

In [5]:
# conv_net = nn.Sequential(
#     nn.Conv2d(1, 28, 5),
#     nn.MaxPool2d(2),
#     nn.ReLU(),

#     nn.BatchNorm2d(28),
#     nn.Dropout2d(0.25),
#     nn.Conv2d(28, 64, 5),

#     nn.MaxPool2d(2),
#     nn.ReLU(),

#     nn.BatchNorm2d(64),
#     nn.Dropout2d(0.25),
#     FlatternLayer()
# )

# # Finally, check the Image-Size by Dummy-Data by Convolution Product.
# test_input = torch.ones(1, 1, 28, 28)
# conv_output_size = conv_net(test_input).size()[-1]

# # 2-Layer MLP
# mlp = nn.Sequential(
#     nn.Linear(conv_output_size, 200),
#     nn.ReLU(),
#     nn.BatchNorm1d(200),
#     nn.Dropout(0.25),
#     nn.Linear(200, 10)
# )

In [6]:
print(train_imgs.classes)

['burrito', 'taco', 'taco_and_burrito']


In [7]:
print(train_imgs.class_to_idx)

{'burrito': 0, 'taco': 1, 'taco_and_burrito': 2}


In [8]:
# Call resnet18 that already completed training.
net = models.resnet18(pretrained=True)
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [9]:
# Excluding all parameters from differential targets.
for p in net.parameters():
    p.requires_grad=False
    
# Chnage Last-Linear-Layer.
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 2)

In [10]:
# Create Model-Training-Function.
def eval_net(net, data_loader, device="cpu"):
    # Invalidation Dropout and BatchNorm
    net.eval()
    print("test : ", net)
    ys = []
    ypreds = []
    
    for x, y in data_loader:
        # Send to the Device that execute calculation by 'to Method'.
        x = x.to(device)
        y = y.to(device)
        
        # Because forward calculations are all here, 
        # the processing required for automatic differentials is set to off 
        # to limit unnecessary calculations.
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
        
    # The results of the prediction of the mini-deployment 
    # unit are grouped together.
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    
    # Predict Prediction-Accuracy
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

In [13]:
def train_net(net, train_loader, test_loader, only_fc=True,
             optimizer_cls=optim.Adam,
             loss_fn=nn.CrossEntropyLoss(),
             n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    
    if only_fc:
        # Delivery Last-Linear-Layer Parameter to optimizer.
        optimizer = optimizer_cls(net.fc.parameters())
    else:
        optimizer = optimizer_cls(net.parameters())
    
    for epoch in range(n_iter):
        running_loss = 0.0
        
        # Set Neural-Network as a Training-Mode.
        net.train()
        n = 0
        n_acc = 0
        
        # It takes a lot of time, 
        # so use tqdm to display Progress-Bar.
        for i, (xx, yy) in enumerate(train_loader):
            total=len(train_loader)
            xx = xx.to(device)
            yy = yy.to(device)
            
            h = net(xx)
            loss = loss_fn(h, yy)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        
        train_losses.append(running_loss / i)
        
        # Prediction-Accuracy of Training-Data.
        train_acc.append(n_acc / n)
        
        # Prediction-Accuracy of Verified-Data.
        val_acc.append(eval_net(net, test_loader, device))
        
        # Display the result of epoch.
        print(epoch, train_losses[-1], train_acc[-1],
             val_acc[-1], flush=True)
        
# Send All of Neural-Network Parameter to GPU or CPU.
# or net.to("cuda:0")
net.to("cpu")

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [17]:
# Execute Training
# train_net(net, train_loader, test_loader, n_iter=20, device="cpu")
# or train_net(net, train_loader, test_loader, n_iter=20, device="cuda:0")